### Outline

* OpenAI function calling
* Langchain Expression Language (LCEL)
* OpenAI function calling in Langchain
* Tagging and extraction using OpenAI function calling
* Tools and Routing
* Conversational Agent


In [10]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# openai.api_key = os.environ['OPENAI_API_KEY']

openai.api_key = "41bad9b287c02b34b4a844bcfc21f18225cc0852f983652c8952908795a75c07" 

TOGETHER_API_KEY = "41bad9b287c02b34b4a844bcfc21f18225cc0852f983652c8952908795a75c07"

In [3]:
import json

# Dummy function to return the same weather
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

* Define a function: TOOL

In [ ]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        
        "parameters": {
            
            "type": "object",
            "properties": {
                
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                
                "unit": {
                    "type": "string",
                    "enum": ["fahrenheit", "celsius"],
                    "description": "Temperature unit"
                }
            },
            
            # Mandatory
            "required": ["location"],
        }
    }
]

In [7]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [ ]:
# Tool Wrapper
from langchain.tools import Tool # Object

# Defining the tool
weather_tool = Tool(
    name="get_current_weather", # Name must match the function name
    # Arguments to pass to the function (lambda, unit) -> to the TOOL
    func = lambda location, unit="fahrenheit": get_current_weather(location, unit),
    description="Get the current weather in a given location",
)

In [29]:
TOGETHER_API_KEY = "41bad9b287c02b34b4a844bcfc21f18225cc0852f983652c8952908795a75c07"

### Agents based functions and Tools

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType # 

In [ ]:
llm = ChatOpenAI(
    openai_api_base="https://api.together.xyz/v1",
    openai_api_key=TOGETHER_API_KEY,
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
)

In [ ]:
agent = initialize_agent(
    tools=[weather_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, # 0 shot (no memory of past) and ReAct (Reasoning + Agent) - Thought and Action and Observation
    verbose=True,
)

response = agent.invoke("What's the weather in New York?")
print(response)




> Entering new AgentExecutor chain...
 Action: get_current_weather
Action Input: New York
Observation: {"location": "New York", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}
Thought: I now know the final answer
Final Answer: The current weather in New York is sunny and windy with a temperature of 72 degrees Fahrenheit.

> Finished chain.
{'input': "What's the weather in New York?", 'output': 'The current weather in New York is sunny and windy with a temperature of 72 degrees Fahrenheit.'}


In [54]:
print(response)

{'input': "What's the weather in New York?", 'output': 'The current weather in New York is sunny and windy with a temperature of 72 degrees Fahrenheit.'}


In [57]:
response["output"]

'The current weather in New York is sunny and windy with a temperature of 72 degrees Fahrenheit.'

In [61]:
agent.agent # agent for the llm

ZeroShotAgent(llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, template="Answer the following questions as best you can. You have access to the following tools:\n\nget_current_weather(location, unit='fahrenheit') - Get the current weather in a given location\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [get_current_weather]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}"), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x74b4003d19a0>, async_client=<openai.resources.chat.completio

In [62]:
agent.agent.llm_chain

LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, template="Answer the following questions as best you can. You have access to the following tools:\n\nget_current_weather(location, unit='fahrenheit') - Get the current weather in a given location\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [get_current_weather]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}"), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x74b4003d19a0>, async_client=<openai.resources.chat.completions.AsyncCompletions obje